## Searching for bouts for a day of alsa recording

In [1]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2022-01-10 12:19:21,273 root         INFO     Running on pouli


In [2]:
from ceciestunepipe.file import bcistructure as et

### Get the file locations for a session (day) of recordings

In [7]:
reload(et)
sess_par = {'bird': 'z_m18m20_21',
           'sess': '2022-01-08'}


exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='alsa')

raw_folder = exp_struct['folders']['alsa']
bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts_ceciestunepipe')

In [8]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/z_m18m20_21',
 'raw': '/mnt/sphere/speech_bci/raw_data/z_m18m20_21/2022-01-08',
 'alsa': '/mnt/sphere/speech_bci/raw_data/z_m18m20_21/2022-01-08/alsa',
 'kwik': '/experiment/z_m18m20_21/alsa/kwik/2022-01-08',
 'processed': '/mnt/sphere/speech_bci/processed_data/z_m18m20_21/2022-01-08/alsa',
 'derived': '/mnt/sphere/speech_bci/derived_data/z_m18m20_21/2022-01-08/alsa',
 'tmp': '/experiment/tmp/tmp',
 'msort': '/experiment/tmp/z_m18m20_21/alsa/msort/2022-01-08',
 'ksort': '/experiment/tmp/z_m18m20_21/alsa/ksort/2022-01-08/',
 'sort': '/mnt/sphere/speech_bci/derived_data/z_m18m20_21/2022-01-08/alsa/'}

### read a session's bouts pandas
- get the location of the file
- load the pandas pickle
- load the search/spectrogram parameters

In [9]:
from ceciestunepipe.util.sound import boutsearch as bs

In [10]:
### load the search/spectrogram parameters
### these functions are necessary to load the pickle files, which is bad.
### need to not write modules/functions in the pickle

# function for getting one channel out of a wave file
def read_wav_chan(wav_path: str, chan_id: int=0) -> tuple:
    s_f, x = wavfile.read(wav_path, mmap=True)
    return s_f, x[:, chan_id]

def sess_file_id(f_path):
    n = int(os.path.split(f_path)[1].split('-')[-1].split('.wav')[0])
    return n

hparams_file_path = os.path.join(bouts_folder, 'bout_search_params.pickle')
with open(hparams_file_path, 'rb') as fh:
    hparams = pickle.load(fh)

In [11]:
#bpd, x, p = bs.get_bouts_in_file(one_wav_path, hparams)

bouts_auto_file_path = os.path.join(bouts_folder, hparams['bout_auto_file'])

#load. It is important to reset index because the manual curation requires unique indexing
bpd = pd.read_pickle(bouts_auto_file_path).reset_index(drop=True)
bpd.head(1)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram
0,1104595,1109110,53020560,53237280,"[0.3332896099066227, 0.3997395187092817, 1.013...",0.387231,8.714867,True,/mnt/sphere/speech_bci/raw_data/z_m18m20_21/20...,4515,"[[1104595, 1104655], [1105845, 1105870], [1105...",22,"[3, 11, 253, 262, 271, 285, 299, 340, 347, 354...",41,110.121951,"[-3, -5, -4, -4, -3, -3, -4, -4, -3, -3, -4, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


###### if it wasnt saved (which is a bad mistake), read the sampling rate from the first file in the session

In [12]:
def sample_rate_from_wav(wav_path):
    x, sample_rate = wavfile.read(wav_path)
    return sample_rate

if hparams['sample_rate'] is None:
    one_wav_path = bpd.loc[0, 'file']
    logger.info('Sample rate not saved in parameters dict, searching it in ' + one_wav_path)
    hparams['sample_rate'] = sample_rate_from_wav(one_wav_path)

In [13]:
def cleanup(bout_pd: pd.DataFrame):
    ## check for empty waveforms (how woudld THAT happen???)
    bout_pd['valid_waveform'] = bout_pd['waveform'].apply(lambda x: (False if x.size==0 else True))
    
    # valid is & of all the validated criteria
    bout_pd['valid'] = bout_pd['valid_waveform']
    
    # drop not valid and reset index
    bout_pd.drop(bout_pd[bout_pd['valid']==False].index, inplace=True)
    bout_pd.reset_index(drop=True, inplace=True)

cleanup(bpd)

#### compute the spectrograms

In [25]:
hparams

{'num_freq': 1024,
 'preemphasis': 0.97,
 'frame_shift_ms': 5,
 'frame_length_ms': 10,
 'min_level_db': -55,
 'ref_level_db': 110,
 'sample_rate': 48000,
 'mel_filter': False,
 'num_mels': 1024,
 'fmin': 300,
 'fmax': 7000,
 'max_iters': 200,
 'griffin_lim_iters': 20,
 'power': 1.5,
 'read_wav_fun': <function __main__.read_wav_chan(wav_path: str, chan_id: int = 0) -> tuple>,
 'file_order_fun': <function __main__.sess_file_id(f_path)>,
 'min_segment': 20,
 'min_silence': 1500,
 'min_bout': 100,
 'peak_thresh_rms': 0.55,
 'thresh_rms': 0.25,
 'mean_syl_rms_thresh': 0.3,
 'max_bout': 5000,
 'l_p_r_thresh': 60,
 'waveform_edges': 1000,
 'bout_auto_file': 'bout_auto.pickle',
 'bout_curated_file': 'bout_checked.pickle'}

In [26]:
display_params = hparams.copy()


{'num_freq': 1024,
 'preemphasis': 0.97,
 'frame_shift_ms': 5,
 'frame_length_ms': 10,
 'min_level_db': -55,
 'ref_level_db': 110,
 'sample_rate': 48000,
 'mel_filter': False,
 'num_mels': 1024,
 'fmin': 300,
 'fmax': 7000,
 'max_iters': 200,
 'griffin_lim_iters': 20,
 'power': 1.5,
 'read_wav_fun': <function __main__.read_wav_chan(wav_path: str, chan_id: int = 0) -> tuple>,
 'file_order_fun': <function __main__.sess_file_id(f_path)>,
 'min_segment': 20,
 'min_silence': 1500,
 'min_bout': 100,
 'peak_thresh_rms': 0.55,
 'thresh_rms': 0.25,
 'mean_syl_rms_thresh': 0.3,
 'max_bout': 5000,
 'l_p_r_thresh': 60,
 'waveform_edges': 1000,
 'bout_auto_file': 'bout_auto.pickle',
 'bout_curated_file': 'bout_checked.pickle'}

In [27]:
display_params = hparams.copy()
display_params['num_freq'] = 2048
display_params['frame_shift_ms'] = 1
display_params['fame_length_ms'] = 20

bpd['spectrogram'] = bpd['waveform'].apply(lambda x: bs.gimmepower(x, display_params)[2])
logger.info('saving bout pandas with spectrogram to ' + bouts_auto_file_path)
bpd.to_pickle(bouts_auto_file_path)

2022-01-10 12:25:47,832 root         INFO     saving bout pandas with spectrogram to /mnt/sphere/speech_bci/derived_data/z_m18m20_21/2022-01-08/alsa/bouts_ceciestunepipe/bout_auto.pickle


In [28]:
bpd

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,1104595,1109110,53020560,53237280,"[0.3332896099066227, 0.3997395187092817, 1.013...",0.387231,8.714867,False,/mnt/sphere/speech_bci/raw_data/z_m18m20_21/20...,4515,"[[1104595, 1104655], [1105845, 1105870], [1105...",22,"[3, 11, 253, 262, 271, 285, 299, 340, 347, 354...",41,110.121951,"[-3, -5, -4, -4, -3, -3, -4, -4, -3, -3, -4, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
1,1140795,1142175,54758160,54824400,"[0.23053957533336494, 0.7976663474412168, 1.53...",0.387231,2.199865,False,/mnt/sphere/speech_bci/raw_data/z_m18m20_21/20...,1380,"[[1140795, 1140855], [1142105, 1142175]]",2,"[3, 8, 268]",3,460.000000,"[-5, -6, -5, -6, -7, -7, -6, -6, -7, -6, -6, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
2,1297705,1298325,62289840,62319600,"[3.4594913163872247, 5.97914886378902, 6.39336...",0.387231,9.404134,False,/mnt/sphere/speech_bci/raw_data/z_m18m20_21/20...,620,"[[1297705, 1297790], [1298250, 1298325]]",2,"[6, 13, 70, 110, 118]",5,124.000000,"[-10, -10, -9, -9, -10, -12, -12, -11, -10, -1...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
3,1479060,1483475,70994880,71206800,"[0.9311107271111801, 0.32773324487697686, 1.10...",0.387231,19.999210,False,/mnt/sphere/speech_bci/raw_data/z_m18m20_21/20...,4415,"[[1479060, 1479130], [1479235, 1479310], [1479...",27,"[5, 10, 43, 73, 81, 112, 121, 141, 150, 174, 1...",63,70.079365,"[1, 2, 2, 1, 0, 0, 2, 1, 0, 1, 1, 1, 1, 0, 0, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
4,1505165,1508580,72247920,72411840,"[0.2610870440821144, 1.6839679608355962, 5.787...",0.387231,9.462058,False,/mnt/sphere/speech_bci/raw_data/z_m18m20_21/20...,3415,"[[1505165, 1505210], [1505220, 1505285], [1505...",25,"[4, 9, 14, 20, 35, 49, 61, 90, 103, 124, 141, ...",52,65.673077,"[1, 3, 3, 1, 0, 1, 4, 3, 1, 0, 1, 2, 2, 1, -1,...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,215195,216430,10329360,10388640,"[2.4416686733876354, 6.326772109743472, 11.199...",0.166606,14.633692,False,/mnt/sphere/speech_bci/raw_data/z_m18m20_21/20...,1235,"[[215195, 215275], [215350, 215430], [216185, ...",4,"[2, 11, 35, 42, 203, 209, 233, 238]",8,154.375000,"[1, 0, 2, 1, 0, 0, 0, 0, -1, -2, -1, -2, -1, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
172,249240,249570,11963520,11979360,"[0.9464954445528272, 1.477033027246781, 0.6695...",0.208386,74.465423,False,/mnt/sphere/speech_bci/raw_data/z_m18m20_21/20...,330,"[[249240, 249275], [249465, 249485], [249550, ...",3,"[4, 36, 47, 62]",4,82.500000,"[-3, -6, -7, -6, -6, -7, -8, -8, -9, -9, -10, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
173,72195,73105,3465360,3509040,"[0.096076576322517, 0.6448550532779609, 1.3344...",0.067692,1.416882,False,/mnt/sphere/speech_bci/raw_data/z_m18m20_21/20...,910,"[[72195, 72250], [73065, 73105]]",2,"[3, 9, 179]",3,303.333333,"[2, 3, 3, 3, 4, 2, 4, 5, 4, 4, 3, 3, 5, 3, 2, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
174,605750,607170,29076000,29144160,"[0.043948993992791494, 0.07737698821344607, 0....",0.067692,0.200895,False,/mnt/sphere/speech_bci/raw_data/z_m18m20_21/20...,1420,"[[605750, 605790], [607150, 607170]]",2,"[1, 6, 281]",3,473.333333,"[-2, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, 0, ...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True


In [29]:
bpd['file'][0]

'/mnt/sphere/speech_bci/raw_data/z_m18m20_21/2022-01-08/alsa/07-00-01-02.wav'

## inspect the bouts and curate them

#### visualize one bout

In [30]:
bpd.iloc[0]

start_ms                                                    1104595
end_ms                                                      1109110
start_sample                                               53020560
end_sample                                                 53237280
p_step            [0.3332896099066227, 0.3997395187092817, 1.013...
rms_p                                                      0.387231
peak_p                                                     8.714867
bout_check                                                    False
file              /mnt/sphere/speech_bci/raw_data/z_m18m20_21/20...
len_ms                                                         4515
syl_in            [[1104595, 1104655], [1105845, 1105870], [1105...
n_syl                                                            22
peaks_p           [3, 11, 253, 262, 271, 285, 299, 340, 347, 354...
n_peaks                                                          41
l_p_ratio                                       

In [31]:
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets

In [33]:
def viz_one_bout(df: pd.Series, sub_sample=1):
    # get the power and the spectrogram
    sxx = df['spectrogram'][:, ::sub_sample]
    x = df['waveform'][::sub_sample]
    
    # the trace
    tr_waveform = go.Scatter(y=x)
    figwidg_waveform = go.FigureWidget(data=[tr_waveform],
                                      layout= {'height': 300,'width':1000})

    # the spectrogram
    fig_spectrogram = px.imshow(sxx, 
                                     labels={}, 
                                     color_continuous_scale='Inferno',
                                    aspect='auto')

    fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
    fig_spectrogram.update_xaxes(showticklabels=False)
    fig_spectrogram.update_yaxes(showticklabels=False)
    
    
    figwidg_spectrogram = go.FigureWidget(fig_spectrogram)
    
    display(widgets.VBox([figwidg_waveform,
              figwidg_spectrogram]))
    

viz_one_bout(bpd.iloc[23])

    'data': [{'type': 'scatter',
              'uid': '631d178f-71a4-4cc6-aeea-b…

In [34]:
bpd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,valid_waveform,valid,spectrogram,confusing
0,1104595,1109110,53020560,53237280,"[0.3332896099066227, 0.3997395187092817, 1.013...",0.387231,8.714867,False,/mnt/sphere/speech_bci/raw_data/z_m18m20_21/20...,4515,"[[1104595, 1104655], [1105845, 1105870], [1105...",22,"[3, 11, 253, 262, 271, 285, 299, 340, 347, 354...",41,110.121951,"[-3, -5, -4, -4, -3, -3, -4, -4, -3, -3, -4, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True
1,1140795,1142175,54758160,54824400,"[0.23053957533336494, 0.7976663474412168, 1.53...",0.387231,2.199865,False,/mnt/sphere/speech_bci/raw_data/z_m18m20_21/20...,1380,"[[1140795, 1140855], [1142105, 1142175]]",2,"[3, 8, 268]",3,460.000000,"[-5, -6, -5, -6, -7, -7, -6, -6, -7, -6, -6, -...",True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",True


## use it in a widget


#### add a 'confusing' label, for not/sure/mixed.
we want to avoid having things we are not sure of in the training dataset

In [35]:
bpd.reset_index(drop=True, inplace=True)

In [36]:
## Set confusing by default, will only be False once asserted bout/or not
bpd['confusing'] = True
bpd['bout_check'] = False

In [37]:
### Create a counter object (count goes 1:1 to DataFrame index)
from traitlets import CInt, link

class Counter(widgets.DOMWidget):
    value = CInt(0)
    value.tag(sync=True)

In [38]:
class VizBout():
    def __init__(self, hparams, bouts_pd):
        self.bout = None
        self.bouts_pd = bouts_pd
        self.bout_series = None
        self.is_bout = None
        self.is_confusing = None
        
        self.bout_counter = None
        self.bout_id = None
        
        self.buttons = {}
        self.m_pick = None
        
        
        self.fig_waveform = None
        self.fig_spectrogram = None
        
        self.figwidg_waveform = None
        self.figwidg_spectrogram = None
        
        self.fig_width = 2
        self.sub_sample = 10
        
        self.x = None
        self.sxx = None
        self.tr_waveform = None
        
        self.s_f = hparams['sample_rate']
        
        self.init_fig()
        self.init_widget()
        self.show()
        
    def init_fig(self):
        # the trace
        self.tr_waveform = go.Scatter(y=np.zeros(500))
        self.figwidg_waveform = go.FigureWidget(data=[self.tr_waveform],
                                               layout={'width': 1000, 'height':300})
        
        # the spectrogram
        self.fig_spectrogram = px.imshow(np.random.rand(500, 500), 
                                         labels={}, 
                                         color_continuous_scale='Inferno',
                                        aspect='auto')
        
        self.fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
        self.fig_spectrogram.update_xaxes(showticklabels=False)
        self.fig_spectrogram.update_yaxes(showticklabels=False)
        self.figwidg_spectrogram = go.FigureWidget(self.fig_spectrogram)
        
        
    def init_widget(self):
        # declare elements
        # lay them out
        #
        
        self.bout_counter = Counter()
        self.is_bout = widgets.Checkbox(description='is bout')
        self.is_confusing = widgets.Checkbox(description='Not sure or mixed')
        
        self.buttons['Next'] = widgets.Button(description="Next", button_style='info',
                                              icon='plus')   
        self.buttons['Prev'] = widgets.Button(description="Prev", button_style='warning',
                                              icon='minus')
        self.buttons['Check'] = widgets.Button(description="Check", button_style='success', 
                                               icon='check')
        self.buttons['Uncheck'] = widgets.Button(description="Uncheck", button_style='danger',
                                                 icon='wrong')
        
        [b.on_click(self.button_click) for b in self.buttons.values()]
        
        left_box = widgets.VBox([self.buttons['Prev'], self.buttons['Uncheck']])
        right_box = widgets.VBox([self.buttons['Next'], self.buttons['Check']])
        button_box = widgets.HBox([left_box, right_box])

        self.m_pick = widgets.IntSlider(value=0, min=0, max=self.bouts_pd.index.size-1,step=1, 
                                        description="Bout candidate index")
        
        
        control_box = widgets.HBox([button_box,
                                  widgets.VBox([self.is_bout, self.is_confusing]),
                                    self.m_pick])
        
        link((self.m_pick, 'value'), (self.bout_counter, 'value'))

        self.update_bout()
        
        self.is_bout.observe(self.bout_checked, names='value')
        self.is_confusing.observe(self.confusing_checked, names='value')
        self.m_pick.observe(self.slider_change, names='value')
        
        all_containers = widgets.VBox([control_box, 
                                       self.figwidg_waveform, self.figwidg_spectrogram])
        display(all_containers)
#         display(button_box)
#         display(self.m_pick)
#         display(self.is_bout)
#         display(self.fig)
        
    def button_click(self, button):        
        self.bout_id = self.bout_counter.value
        curr_bout = self.bout_counter
        
        if button.description == 'Next':
            curr_bout.value += 1
        elif button.description == 'Prev':
            curr_bout.value -= 1
        elif button.description == 'Check':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        elif button.description == 'Uncheck':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = False
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        
        # handle the edges of the counter
        if curr_bout.value > self.m_pick.max:
            curr_bout.value = 0
            
        if curr_bout.value < self.m_pick.min:
            curr_bout.value = self.m_pick.max
    
    def slider_change(self, change):
        #logger.info('slider changed')
        #self.bout_counter = change.new
        #clear_output(True)
        self.update_bout()
        self.show()
            
    def bout_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'bout_check'] = bc['new']
    
    def confusing_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'confusing'] = bc['new']
    
    def update_bout(self):
        self.bout_id = self.bout_counter.value
        self.bout_series = self.bouts_pd.iloc[self.bout_id]
        
        self.is_bout.value = bool(self.bout_series['bout_check'])
        self.is_confusing.value = bool(self.bout_series['confusing'])
        
        self.x = self.bout_series['waveform'][::self.sub_sample]
        self.sxx = self.bout_series['spectrogram'][::self.sub_sample]
        
    def show(self):
        #self.fig.clf()
        #self.init_fig()
        # update
#         self.update_bout()
        #plot
        #logger.info('showing')
        
        # Show the figures
        with self.figwidg_waveform.batch_update():
            self.figwidg_waveform.data[0].y = self.x
            self.figwidg_waveform.data[0].x = np.arange(self.x.size) * self.sub_sample / self.s_f 
            
        with self.figwidg_spectrogram.batch_update():
            self.figwidg_spectrogram.data[0].z = np.sqrt(self.sxx[::-1])
            
        

viz_bout = VizBout(hparams, bpd)

In [39]:
np.where(viz_bout.bouts_pd['bout_check']==True)[0].size

33

### save it

In [40]:
hparams

{'num_freq': 1024,
 'preemphasis': 0.97,
 'frame_shift_ms': 5,
 'frame_length_ms': 10,
 'min_level_db': -55,
 'ref_level_db': 110,
 'sample_rate': 48000,
 'mel_filter': False,
 'num_mels': 1024,
 'fmin': 300,
 'fmax': 7000,
 'max_iters': 200,
 'griffin_lim_iters': 20,
 'power': 1.5,
 'read_wav_fun': <function __main__.read_wav_chan(wav_path: str, chan_id: int = 0) -> tuple>,
 'file_order_fun': <function __main__.sess_file_id(f_path)>,
 'min_segment': 20,
 'min_silence': 1500,
 'min_bout': 100,
 'peak_thresh_rms': 0.55,
 'thresh_rms': 0.25,
 'mean_syl_rms_thresh': 0.3,
 'max_bout': 5000,
 'l_p_r_thresh': 60,
 'waveform_edges': 1000,
 'bout_auto_file': 'bout_auto.pickle',
 'bout_curated_file': 'bout_checked.pickle'}

In [41]:
### get the curated file path
##save to the curated file path
bouts_curated_file_path = os.path.join(bouts_folder, hparams['bout_curated_file'])

viz_bout.bouts_pd.to_pickle(bouts_curated_file_path)
logger.info('saved curated bout pandas to pickle {}'.format(bouts_curated_file_path))

2022-01-10 12:32:36,987 root         INFO     saved curated bout pandas to pickle /mnt/sphere/speech_bci/derived_data/z_m18m20_21/2022-01-08/alsa/bouts_ceciestunepipe/bout_checked.pickle


In [ ]:
viz_bout.bouts_pd['file'][0]

In [ ]:
viz_bout.bouts_pd.head(5)